In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# raw data

genes = pd.read_excel('../data/pdx/List of Genes Differentially Expressed upon Different Treatments.xlsx')
genes = genes.iloc[:,[1,2,4,5,7,8]]
genes.columns = ['DHTup', 'DHTdown', 'E2up', 'E2down', 'P4up', 'P4down']
genes = genes.iloc[2:,:]
genes.info()
genes


In [ ]:
# using pre-processed gene list (sorted and without duplicates)

path = '../data/pdx/List of Genes Differentially Expressed upon Different Treatments.xlsx'
geneList = pd.read_excel(path, sheet_name='genesListed')
geneList.info()
geneList

In [ ]:
# extracting pdx tumor data

path = '../data/pdx/Human_matrix_DESEQ2normalized_removedlowlyexpressedgenes.xlsx'
init = pd.read_excel(path)
tumor = init
tumor = tumor.drop(['ensgene', 'biotype', 'description'], axis = 1)
tumor = tumor.transpose()
tumor.columns = tumor.iloc[0,:]
tumor = tumor.iloc[1:,:]
tumor = tumor.astype('float64')
tumor.describe()

In [ ]:
# select only data of genes from list

selectedGenes = geneList.Genes
selectionPDX = tumor[selectedGenes]
selectionPDX

In [ ]:
# PCA decomposition of original gene list
# we want to verify that the pre-selected genes are linearly independent

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = decomposition.PCA()
pca.fit(selectionPDX)  
PCA(copy=True, iterated_power='auto', n_components=None, random_state=None, svd_solver='auto', tol=0.0, whiten=False)

# Explained variance by PCA
#print(pca.explained_variance_)

# Singular values of PCA
#print(pca.singular_values_)

y_pos = np.arange(len(pca.singular_values_))
plt.bar(y_pos, pca.singular_values_, align='center', alpha=0.5)
plt.ylabel('Values')
plt.xlabel('Principal components')
plt.title('PCA - Singular values')
plt.show()

# 
#pca.n_components = 2
#X_reduced = pca.fit_transform(X)
#X_reduced.shape